# Trading Game tournament

This notebook runs a tournament for an asymmetric trading game, using the Axelrod library.

The game consists of two players, a trader $\mathcal{T}$ and a regulator $\mathcal{R}$. 

The trader can either do a regular trade, $\mathcal{T}_r$ which has a small reward and no risk of being caught by the regulator, or an inside trade $\mathcal{T}_I$ which has a much higher reward but, if caught by the regulator, will lead to harsh penalties.

The regulator can choose not to regulate $\mathcal{R}_n$, which has a small benefit if no inside trading takes place (as they're doing their job properly) and a penalty if inside trading is taking place (as inside trading has been detected, but they don't know who's doing it!). Or, they can regulate $\mathcal{R}_r$, which has a financial cost if done when unnecessary, but a reward if they catch an inside trader.

In [ ]:
import axelrod as axl
import numpy as np

# local files
import trade_tournament as tt
import trade_strategy as ts
from asymmetric_game import AsymmetricGame

In [ ]:
T = np.array([
    [1, 1],
    [6, -10]
    ])

R = np.array([
    [1, -1],
    [-3, 5]
    ])

trade_game = AsymmetricGame(T, R)

In this cell, input trading strategies into the `traders` list, and regulation strategies into the `regulators` list. Some of these are from the Axelrod library, and others are from the file trade_strategy.py in this folder.

In [ ]:
traders = [axl.Cooperator(), axl.TitFor2Tats(), ts.Careful(), axl.Random(0.75), ts.NceBitten(1), ts.NceBitten(5)]
regulators = [ts.SuspiciousForgiving(), ts.SuspiciousGrudge(), axl.Random(0.5), axl.Random(0.2), ts.Grudge()]

Finally, run this block to run the tournament.

In [ ]:
tt.trade_tournament(traders, regulators, trade_game)

We additionally can use this game to simulate the "paired Moran process", which works as follows:

Let $P_1$, $P_2$ be two populations; $P_1$ of row players, and $P_2$ of column players. Then the paired Moran process algorithm begins:

1. Calculate the fitness of each member of each population against the *other* population. For example, for the Hawk-Dove game if $P_1$ = {2 hawk, 1 dove} and $P_2$ = {1 hawk, 2 dove}, the fitness of the dove in $P_1$ is calculated based on the game outcomes against the 1 hawk and 2 doves in $P_2$.

2. Birth and death for each population is then calculated from these fitnesses in the same way as the regular Moran process. This is done independently for each population (e.g. 1 birth and 1 death in $P_1$, *and* 1 birth and 1 death in $P_2$)

3. Repeat until each population are homogeneous.

This allows us to use the Moran process to study emergent behaviour where competition between *every* member of a population is unfeasible - for example, in our application, the traders are not playing between themselves, and neither are the regulators; furthermore, the best strategies for trading vs regulating will not be the same strategy (as they have different payoff matrices!)

We simulate it by running an Axelrod tournament at each step, and then using that for each players' utilities.

In [ ]:
import paired_moran as pm

pm.paired_moran(traders, regulators, trade_game)